In [ ]:
%%capture
!pip install langchain langchain-community langchain-huggingface faiss-cpu groq langchain-groq

In [ ]:
import os
from google.colab import userdata

In [ ]:
os.environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
# Load the pre-built FAISS vector database and initialize the LLM (Llama-3 via Groq)
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)


In [ ]:
path_local = "faiss_pharmacy"

In [ ]:
vectorstore = FAISS.load_local(
    path_local,
    embeddings,
    allow_dangerous_deserialization=True
)

✅ VectorStore cargada correctamente desde archivos locales.


In [ ]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0,
)

✅ Llama-3-70B listo para razonar.


In [ ]:
# Define the RAFT prompt: enforces step-by-step reasoning (Chain of Thought) based strictly on the provided medical context

from langchain_core.prompts import ChatPromptTemplate

raft_generation_prompt = ChatPromptTemplate.from_template("""
You are a Pharmacology professor creating a high-difficulty exam.
Use the provided CONTEXT to create one question and a detailed answer.

FORMAT RULES:
1. Start directly with "QUESTION:" followed by the created question.
2. Then write "ANSWER:" followed by the response.
3. The answer must be detailed, showing a step-by-step reasoning (Chain of Thought)
and ending with a conclusion, but EVERYTHING must flow naturally without using
section headers like "Step 1" or "Introduction".
4. Base the answer strictly on the provided context.

CONTEXTO:
{context}

Create a question and an answer based strictly on this context.
""")

def generate_raft_example(query):

    docs = vectorstore.similarity_search(query, k=3)
    context_text = "\n\n".join([d.page_content for d in docs])

    chain = raft_generation_prompt | llm
    response = chain.invoke({"context": context_text})

    return response.content

print(generate_raft_example("What is the mechanism of action of Ibuprofen?"))

QUESTION: What are the key differences in the pharmacological profiles of flurbiprofen and ibuprofen, including their indications, pharmacokinetics, and adverse effect profiles, and how do these differences impact their clinical use?

ANSWER: Flurbiprofen and ibuprofen are both arylpropionic acid derivatives that inhibit the production of prostaglandins via inhibition of COX, resulting in analgesia, antipyresis, and anti-inflammatory effects. However, they have distinct pharmacological profiles that set them apart. Flurbiprofen is available in various formulations, including topical ophthalmic and intravenous forms, and is effective for perioperative analgesia in minor ear, neck, and nose surgery, as well as in lozenge form for sore throat. In contrast, ibuprofen is often prescribed in lower doses, typically less than 2400 mg daily, at which it has analgesic but not anti-inflammatory efficacy, and is available over the counter in low-dose forms. Ibuprofen is also effective in closing p

In [ ]:
import json
import time
import pandas as pd
import random

# Load medicine names from the CSV (Ensure the filename is correct)
df = pd.read_csv('medicine_df.csv')
med_list = df['Medicine Name'].unique().tolist()

# Function to save each example
def append_to_jsonl(data, filename="pharmacy_raft_dataset.jsonl"):
    with open(filename, 'a', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False)
        f.write('\n')

# Bulk generation loop
count = 0
target = 100

print(f" Generating {target} RAFT examples.")

while count < target:
    try:
        medicine = random.choice(med_list)
        print(f"[{count+1}/{target}] Processing: {medicine}")

        # Generamos el contenido con el prompt en inglés
        raw_output = generate_raft_example(f"Mechanism, uses and side effects of {medicine}")

        # Separate Question and Answer for the dataset
        if "QUESTION:" in raw_output and "ANSWER:" in raw_output:
            parts = raw_output.split("ANSWER:")
            question = parts[0].replace("QUESTION:", "").strip()
            answer = parts[1].strip()

            # Retrieve documents for the RAFT 'context' field
            docs = vectorstore.similarity_search(medicine, k=3)
            context = "\n\n".join([d.page_content for d in docs])

            # Save
            example = {
                "instruction": question,
                "context": context,
                "answer": answer
            }
            append_to_jsonl(example)
            count += 1

        # Pause to avoid Groq Rate Limits
        time.sleep(1.5)

    except Exception as e:
        print(f"⚠️ Error: {e}. Retrying...")
        time.sleep(5)

print("\nFinished")

🚀 Generating 100 RAFT examples. This will take a while...
[1/100] Processing: Ranidom-O Oral Suspension Orange Sugar Free
[2/100] Processing: Xpect-B Expectorant Mango Payari Black Currant
[3/100] Processing: Halotop-S Lotion
[4/100] Processing: Xyzal 10mg Tablet
[5/100] Processing: Melanorm-MS Cream
[6/100] Processing: Rantac Infant Syrup Mint
[7/100] Processing: Olmezest CH 40 Tablet
[8/100] Processing: Superspas Oral Gel Orange Sugar Free
[9/100] Processing: Tynept OD Tablet ER
[10/100] Processing: Distaclor 187mg Oral Suspension Strawberry
[11/100] Processing: Rosvin 10 Tablet
[12/100] Processing: Syncapone 100 Tablet
[13/100] Processing: Gestofit 100 Soft Gelatin Capsule
[14/100] Processing: Zincoderm Cream
[15/100] Processing: Eglucent Mix 50 Kwikpen 100IU/ml
[16/100] Processing: Sizodon MD 0.5 Tablet
[17/100] Processing: Macsart CH 40 Tablet
[18/100] Processing: Glyciphage 250 Tablet
[19/100] Processing: Lutriben Cream
[20/100] Processing: Ozomet-G1 Tablet PR
[21/100] Processing